<a href="https://colab.research.google.com/github/argonism/TsukurinagaraRL/blob/master/Zerokara_chap6_Dueling_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dueling Network
状態価値であるV(s)を使ったもの。
行動によらず、次の状態が決まってしまうような状態がある。
つまり、Q関数が持つ情報は状態Sだけで決まる部分と、行動次第で決まる部分がある。

これを分離しようとしたのがDueling Network

V(s)とA(s, a)に分けて学習し、最後に二つを足してQ(s,a)を求める。

Q. DQNに比べて V(s)につながるネットワークの結合パラメータが行動aによらず毎step学習できるらしいが、まだよくわからん。行動の選択肢が増えるほど、大きな利点になるらしい。
- 行動によらずに、どうやってV(s)を学習するんや。
- ニューラルネットの中で、途中で分岐するのか。


In [ ]:
# 使用するパッケージのインストール
# gym==0.17.2 pyvirtualdisplay==1.3.2
# xvfb=2:1.19.6-1ubuntu4.4 python-opengl=3.1.0+dfsg-1 ffmpeg=7:3.4.8-0ubuntu0.2
# JSAnimation==0.1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install JSAnimation > /dev/null 2>&1

In [ ]:
 import numpy as np
 import matplotlib.pyplot as plt
 %matplotlib inline
 import gym
from gym.wrappers import Monitor

In [ ]:
# 動画の描画関数の宣言
import glob
import io
import os
import base64
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython import display as ipythondisplay
from IPython.display import HTML
from pyvirtualdisplay import Display

display = Display(visible=0, size=(640, 400))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
def reset_video():
  mp4list = glob.glob('video/*.mp4')
  for mp4 in mp4list:
    os.remove(mp4)

def wrap_env(env):
  env = Monitor(env, './video', force=True, video_callable=(lambda ep: ep % 10 == 0))
  reset_video()
  return env

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self, n_in, n_mid, n_out):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(n_in, n_mid)
    self.fc2 = nn.Linear(n_mid, n_out)
    self.fc3_adv = nn.Linear(n_mid, n_out)
    self.fc3_v = nn.Linear(n_mid, 1)

  def forward(self, x):
    h1 = F.relu(self.fc1(x))
    h2 = F.relu(self.fc2(h1))
    adv = self.fc3_adv(h2)
    val = self.fc3_v(h2).expand(-1, adv.size(1))
    # val はadvと足し算するために、サイズを[minibatchx1]から[minibatchx2(n_out)]にexpandする。

    # val + adv - mean(adv)
    output = val + adv - adv.mean(1, keepdim=True).expand(-1, adv.size(1))

    return output

## Q
advが行動を、valが状態価値を意味するわけじゃないというか、
特に状態価値をあらわすようにvalがあるわけじゃなくないか？

つまり、なんとなく状態価値っぽくネットワークを分岐させてるけど、それが状態価値をあらわすとは限らなくない？
Advantage関数も同様に。

結局Qが出てくるだけだから...うーん...

Qiitaみてたらやっぱりそうっぽいようなことが書いてあったのでやっぱり、これらが状態価値やアドバンテージ関数を表してるとは、必ずしも言えないらしい。やったぜ。
